# Coffeescout
This notebook reflects the content of:

* [1] Riding the Storm in a Probabilistic Model Checking Landscape by Hensel, Junges, Quatmann and Volk, Section 4.


## Optimal bias to find a coffee place
Members of the **Random Walk Circle** may approximate a city as a two-dimensional grid and argue that randomly moving through town will ensure that they find coffee.  
One member of the Random Walk Circle, we call them JPK, adores random walks but wants to use a probabilistic model checker to optimize the bias of the four-side die that he will use for moving through town.

### The (parametric) MC.
We model JPK as an agent which has a latitude  ($x$) and a longitude ($y$), and an orientation in a grid-like city with streets in a perfect grid shape and all roads going either north-south or east-west. The town is bordered by four canals that cannot be crossed.  
At every crossing, JPK decides to go with likelihood $p_W$ west, with likelihood $p_E$ east, with likelihood $p_N$ north and with likelihood $p_S$ south. 
JPK initially starts at a hotel quietly located far from the conference venue. It is known that there are two coffee places of sufficient quality in town, named **Cotton & Coffee** (CC) and **Katoen & Koffie** (KK). These coffee places must be on one of the main squares and based on prior knowledge, JPK has a distribution over their locations. The result is a parametric MC (pMC), as depicted partially in [1,Figure 10]. We want to highlight that pMCs models that use rational functions as transition probabilities are somewhat rare in the literature: The use of polynomials is more standard.



Below, we first create the model from the corresponding Prism program.

In [2]:
import stormpy
import stormpy.pars
prism_program = stormpy.parse_prism_program("coffeescout.nm")
options = stormpy.BuilderOptions(True, True)
options.set_build_state_valuations()
pmc = stormpy.build_sparse_parametric_model_with_options(prism_program, options)
print(pmc)

-------------------------------------------------------------- 
Model type: 	DTMC (sparse)
States: 	145
Transitions: 	484
Reward Models:  (default)
State Labels: 	3 labels
   * deadlock -> 0 item(s)
   * init -> 1 item(s)
   * foundcoffee -> 8 item(s)
Choice Labels: 	none
-------------------------------------------------------------- 



The model contains 145 states: There are 36 cells and there are four configurations for where the coffeeplaces could be, which makes 144 states. There is one initial state that randomly determines the position of the coffeeplace. 

The model is parametric, and we can collect the parameters in this model:

In [3]:
parameters = pmc.collect_probability_parameters()
named_parameters = { x.name : x for x in parameters }
print(named_parameters)

{'pN': <Variable pN [id = 8]>, 'pE': <Variable pE [id = 9]>, 'pS': <Variable pS [id = 10]>, 'pW': <Variable pW [id = 7]>}


### Model checking an instantiation
Now, we first instantiate the model to see what happens if we go uniformly in every direction. First, we define the property that we are interested in.

In [22]:
properties = stormpy.parse_properties("R=? [F \"foundcoffee\"]; P=? [F<=20 \"foundcoffee\"]")

Then, we create an instantiator that can replace parameters by concrete values efficiently. We use that one to obtain an instantiated, parameter-free, MC.

In [23]:
instantiator = stormpy.pars.PDtmcInstantiator(pmc)
point = { x : stormpy.RationalRF(1) for x in parameters } # stormpy.RationalRF is the type of constants in parametric MCs in  storm.
instantiated_pmc = instantiator.instantiate(point)


In [24]:
result = stormpy.model_checking(instantiated_pmc, properties[0]).at(instantiated_pmc.initial_states[0])
print(result)
result = stormpy.model_checking(instantiated_pmc, properties[1]).at(instantiated_pmc.initial_states[0])
print(result)


41.384160789515825
0.32786775523823625


Indeed, it takes roughly 41.4 steps on average to get to a coffee place, and the probability to reach the coffee within 20 steps is 0.32.

### Optimal feasibility in the pMC
To analyse whether we can improve the expected time to arrive at the coffee place, we search for different coin-biases. In particular, we are interested in any combination of biases between 0.05 and 0.95. We call this set of biases the parameter region, which we create from a string representation below.

In [6]:
region = stormpy.pars.ParameterRegion.create_from_string("1/20<=pN<=19/20,1/20<=pS<=19/20,1/20<=pE<=19/20,1/20<=pW<=19/20", parameters)

1/20<=pW<=19/20,1/20<=pN<=19/20,1/20<=pE<=19/20,1/20<=pS<=19/20;


Next, we create a region checker that uses PLA. Region checkers require a particular property that we analyse. Here, we first analyse the expected number of steps to get coffee. The region checker (currently) outputs some warnings that can, in this case, safely be ignored. 

In [33]:
region_model_checker = stormpy.pars.create_region_checker(stormpy.Environment(), pmc, properties[0].raw_formula)

 WARN (parameterlifting.h:41): The input model contains a non-linear polynomial as transition: '(pW)/(pS+pE+pN+pW)'. Can not validate that parameter lifting is sound on this model.
 WARN (region.h:139): Could not validate whether parameter lifting is applicable. Please validate manually...


We now use PLA to find an instantiation that minimises the expected number of steps. This is not the exact minimum, but rather an instantiation that induces a value that is close to optimal, up to a user-controlled precision The output is the point and the associated value. 

In [34]:
precision = stormpy.RationalRF(0.1)
result = region_model_checker.compute_extremum(stormpy.Environment(), region, stormpy.OptimizationDirection.Minimize,  precision)
print(float(result[0].constant_part()))
print(result[1])

15.86604815188934
{<Variable pW [id = 7]>: <Rational  (cln)137/2560>, <Variable pN [id = 8]>: <Rational  (cln)31/512>, <Variable pE [id = 9]>: <Rational  (cln)49/512>, <Variable pS [id = 10]>: <Rational  (cln)821/2560>}


We can do the same thing for the second property.

In [31]:
region_model_checker = stormpy.pars.create_region_checker(stormpy.Environment(), pmc, properties[1].raw_formula)
result = region_model_checker.compute_extremum(stormpy.Environment(), region, stormpy.OptimizationDirection.Maximize, precision)
print(float(result[0].constant_part()))
print(result[1])

 WARN (parameterlifting.h:41): The input model contains a non-linear polynomial as transition: '(pW)/(pS+pE+pN+pW)'. Can not validate that parameter lifting is sound on this model.
 WARN (region.h:139): Could not validate whether parameter lifting is applicable. Please validate manually...


NameError: name 'precision' is not defined

## Optimal feasibility with a randomly moving adversary
The next day, JPK knows where the coffee spots are. He would like to enjoy his latte while answering emails and therefore aims to arrive at the coffee spot without bumping in some colleagues that are roaming through town. 
Under these new circumstances, the optimal parameter values cannot ensure a probability of $0.35$ to reach the coffee spot within 20 steps. We are unhappy with this outcome and decide to no longer use a memoryless random walk. 

***TODO*** The reasoning here has not been integrated in the notebook yet.

## Secretly getting coffee
Thus, Storm is now used to compute a policy that uses memory. However, this policy cannot depend on the (unknown) location of the colleague: We must analyse a partially observable MDP (POMDP).

### The partially observable MDP.
Similar to above, the agent has a location $x$, $y$, as has the colleague.
The locations of both coffee places are now fixed and known. The colleague starts in a hotel one block to the south and one block to the east.
The colleague moves randomly but fast: They will move uniformly either one or two steps in any cardinal direction. (Note that this differs from the original text in [1] --- The files for the formulation in [1] are also given in this directory.). The colleague will spot JPK if they are on the same road (either north-south or east-west) and at most two blocks apart. The goal now is to reach a coffee place without being spotted by this colleague.


In [10]:
import stormpy
import stormpy.pomdp
prism_program = stormpy.parse_prism_program("coffeescout-po-adv2.nm")
build_options = stormpy.BuilderOptions(True, True)
build_options.set_build_state_valuations()
build_options.set_build_choice_labels()
pomdp = stormpy.build_sparse_model_with_options(prism_program, build_options)
properties = stormpy.parse_properties("Pmax=? [\"notbad\" U \"foundcoffee\"]")
print(pomdp)

-------------------------------------------------------------- 
Model type: 	POMDP (sparse)
States: 	1297
Transitions: 	20161
Choices: 	5185
Observations: 	73
Reward Models:  (default)
State Labels: 	4 labels
   * deadlock -> 0 item(s)
   * init -> 1 item(s)
   * notbad -> 1141 item(s)
   * foundcoffee -> 72 item(s)
Choice Labels: 	5 labels
   * setprior -> 1 item(s)
   * north -> 1296 item(s)
   * south -> 1296 item(s)
   * west -> 1296 item(s)
   * east -> 1296 item(s)
-------------------------------------------------------------- 



The model contains 36 * 36 + 1 states (36 locations of JPK, 36 locations of adv). In every of the 36 * 36 states, it is possible to go in any of the 4 cardinal directions. In 2 * 36 states, JPK has found coffee.

In [13]:
pomdp = stormpy.pomdp.make_canonic(pomdp)
options = stormpy.pomdp.BeliefExplorationModelCheckerOptionsDouble(True, True)
options.refine_step_limit = 1
options.refine = True
belmc = stormpy.pomdp.BeliefExplorationModelCheckerDouble(pomdp, options)
result = belmc.check(properties[0].raw_formula, [])
print(f"{result.lower_bound}, {result.upper_bound}\n")

AttributeError: module 'stormpy' has no attribute 'pomdp'

### Bounding the number of steps
At the time of writing, Storm does not support (resource/step)-bounded properties. To get insight into the length of the path that JPK walks, we unroll the POMDP by a number of steps, inside the Prism program. Consequentially, the POMDP is bigger.

In [ ]:
prism_program = stormpy.parse_prism_program("coffeescout-po-counter-adv2.nm")
prism_program, properties = stormpy.preprocess_symbolic_input(prism_program, properties, "MAXSTEPS=7")
pomdp = stormpy.build_sparse_model_with_options(prism_program, build_options)
pomdp = stormpy.pomdp.make_canonic(pomdp)
belmc = stormpy.pomdp.BeliefExplorationModelCheckerDouble(pomdp, options)
result = belmc.check(properties[0].raw_formula, [])
print(result.lower_bound)
print(result.upper_bound)

We manually analysed the corresponding policies by manual inspection of the policies. 